In [1]:
'''
20251127:
    load llama2的语言神经元&计算ppl
'''

'\n20251127:\n    load llama2的语言神经元&计算ppl\n'

In [2]:
!pip install datasets

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [4]:
!pip install transformers

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 12.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 8.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 30.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.1.6
    Uninstalling huggingface_hub-1.1.6:
      Successfully uninstalled huggingface_hub-1.1.6


In [6]:
!pip install accelerate

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 3.8 MB/s eta 0:00:00a 0:00:01


In [1]:
import transformers


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

import util
import importlib

importlib.reload(util)      # 只能 reload 模块本身
from util import calc_ppl, get_test_data, get_open_ended_answer, get_open_ended_answer_vllm   # reload 后再重新 import 函数

import copy
import datasets

In [2]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='cuda'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

In [3]:




def get_mask_neuron_model(model, activation_mask_path, need_lang):

    if activation_mask_path:
        activation_masks = torch.load(activation_mask_path)
    else:
        activation_masks = [None]
    
    final_output = []
    if is_llama:
        languages = ["en", "zh", "fr", "es", "vi", "id", "ja"]
    else:
        languages = ["en", "zh", "fr", "es", "vi", "id"]
    
    need_lang = [need_lang]#['en', 'zh', 'vi']
    
    for activation_mask, mask_lang in zip(activation_masks, languages):
    
        if mask_lang not in need_lang:continue
            
        print(f'get mask =====lang:{mask_lang}=====')
    
        
        if activation_mask:
            def factory(mask):
                def llama_forward_org(self, x):
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    activation.index_fill_(2, mask, 0)
                    x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    return x
                def llama_forward(self, x):
                    gate_ = self.gate_proj(x)  # b, l, 2i
                    i = gate_.size(-1)
                    activation = F.silu(gate_)
                    activation.index_fill_(2, mask, 0)
                    x = activation * self.up_proj(x)
                    x = self.down_proj(x)
                    return x
    
                def bloom_forward(self, x: torch.Tensor):
                    x, _ = self.dense_h_to_4h(x)
                    x = self.gelu_impl(x)
                    x.index_fill_(2, mask, 0)
                    x, _ = self.dense_4h_to_h(x)
                    return x
    
                if is_llama:
                    return llama_forward
                else:
                    return bloom_forward
    
            for i, layer_mask in enumerate(activation_mask):
                if is_llama:
                    obj = model.model.layers[i].mlp
                else:
                    obj = model.model.layers[i].mlp
                obj.forward = MethodType(factory(layer_mask.to('cuda')), obj)
    return model








def get_mask_neuron_model_vllm(model, activation_mask_path, need_lang):

    if activation_mask_path:
        activation_masks = torch.load(activation_mask_path)
    else:
        activation_masks = [None]
    
    final_output = []
    if is_llama:
        languages = ["en", "zh", "fr", "es", "vi", "id", "ja"]
    else:
        languages = ["en", "zh", "fr", "es", "vi", "id"]
    
    need_lang = [need_lang]#['en', 'zh', 'vi']
    
    for activation_mask, mask_lang in zip(activation_masks, languages):
    
        if mask_lang not in need_lang:continue
            
        print(f'get mask =====lang:{mask_lang}=====')
    
        
        if activation_mask:
            def factory(mask):
                def llama_forward_org(self, x):
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    activation.index_fill_(2, mask, 0)
                    x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    return x
                def llama_forward(self, x):
                    gate_ = self.gate_proj(x)  # b, l, 2i
                    i = gate_.size(-1)
                    activation = F.silu(gate_)
                    activation.index_fill_(2, mask, 0)
                    x = activation * self.up_proj(x)
                    x = self.down_proj(x)
                    return x
    
                def bloom_forward(self, x: torch.Tensor):
                    x, _ = self.dense_h_to_4h(x)
                    x = self.gelu_impl(x)
                    x.index_fill_(2, mask, 0)
                    x, _ = self.dense_4h_to_h(x)
                    return x
    
                if is_llama:
                    return llama_forward
                else:
                    return bloom_forward
    
            for i, layer_mask in enumerate(activation_mask):
                if is_llama:
                    obj = model.llm_engine.model_executor.driver_worker.model_runner.model.model.layers[i].mlp
                else:
                    obj = model.llm_engine.model_executor.driver_worker.model_runner.model.transformer.h[i].mlp
                obj.forward = MethodType(factory(layer_mask.to('cuda')), obj)
    return model

In [4]:
import os
os.environ["VLLM_ENABLE_V1_MULTIPROCESSING"] = "0"

In [5]:
import argparse
from types import MethodType

import numpy as np
import torch
import torch.nn.functional as F
from vllm import LLM, SamplingParams
from tqdm import tqdm



model_path ='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf'
activation_mask_path='/root/autodl-fs/Language-Specific-Neurons/LLaMA-2-7B.neuron.pth'

is_llama = bool(model_path.find('llama') >= 0)
#model, tokenizer = load_model(model_path)

# 初始化 vLLM
llm = LLM(model=model_path, tensor_parallel_size=1)




INFO 12-01 16:46:40 [utils.py:253] non-default args: {'disable_log_stats': True, 'model': '/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf'}
WARNING 12-01 16:46:40 [model.py:438] The global random seed is set to 0. Since VLLM_ENABLE_V1_MULTIPROCESSING is set to False, this may affect the random state of the Python process that launched vLLM.
INFO 12-01 16:46:40 [model.py:631] Resolved architecture: LlamaForCausalLM
INFO 12-01 16:46:40 [model.py:1745] Using max model len 4096
INFO 12-01 16:46:40 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 12-01 16:46:40 [core.py:93] Initializing a V1 LLM engine (v0.11.2) with config: model='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf', speculative_config=None, tokenizer='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=Non

Loading pt checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 12-01 16:47:39 [default_loader.py:314] Loading weights took 56.55 seconds
INFO 12-01 16:47:39 [gpu_model_runner.py:3338] Model loading took 12.5524 GiB memory and 56.828024 seconds
INFO 12-01 16:47:43 [backends.py:631] Using cache directory: /root/.cache/vllm/torch_compile_cache/e5e105181a/rank_0_0/backbone for vLLM's torch.compile
INFO 12-01 16:47:43 [backends.py:647] Dynamo bytecode transform time: 4.09 s
INFO 12-01 16:47:46 [backends.py:210] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.931 s
INFO 12-01 16:47:47 [monitor.py:34] torch.compile takes 6.02 s in total
INFO 12-01 16:47:48 [gpu_worker.py:359] Available KV cache memory: 14.89 GiB
INFO 12-01 16:47:48 [kv_cache_utils.py:1229] GPU KV cache size: 30,480 tokens
INFO 12-01 16:47:48 [kv_cache_utils.py:1234] Maximum concurrency for 4,096 tokens per request: 7.44x


2025-12-01 16:47:48,567 - INFO - autotuner.py:256 - flashinfer.jit: [Autotuner]: Autotuning process starts ...
2025-12-01 16:47:48,578 - INFO - autotuner.py:262 - flashinfer.jit: [Autotuner]: Autotuning process ends
Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:02<00:00, 22.95it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 34.22it/s]


INFO 12-01 16:47:52 [gpu_model_runner.py:4244] Graph capturing finished in 4 secs, took -0.59 GiB
INFO 12-01 16:47:52 [core.py:250] init engine (profile, create kv cache, warmup model) took 12.84 seconds
INFO 12-01 16:47:52 [llm.py:352] Supported tasks: ('generate',)


## test ppl

In [6]:
# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (None, 'mask_en'),
    (None, 'mask_vi'),
    (None, 'mask_zh'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    if model_name != 'org_model':
        i_model = get_mask_neuron_model(model, activation_mask_path, model_name.split('_')[-1])

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:53<00:00, 18.75it/s]


ppl: tensor(11.5736, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:21<00:00,  7.09it/s]


ppl: tensor(14.1997, device='cuda:0')
data name: zh_data


...

ppl: tensor(11.3950, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:20<00:00,  7.10it/s]


ppl: tensor(13.4718, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [01:39<00:00, 10.01it/s]

ppl: tensor(10.2067, device='cuda:0')


In [8]:
# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    #(None, 'mask_en'),
    #(None, 'mask_vi'),
    (None, 'mask_zh'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    if model_name != 'org_model':
        i_model = get_mask_neuron_model(model, activation_mask_path, model_name.split('_')[-1])

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:52<00:00, 18.93it/s]


ppl: tensor(11.5736, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:21<00:00,  7.09it/s]


ppl: tensor(14.1997, device='cuda:0')
data name: zh_data


...

ppl: tensor(11.3950, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:20<00:00,  7.10it/s]


ppl: tensor(13.4718, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [01:39<00:00, 10.01it/s]

ppl: tensor(10.2067, device='cuda:0')


## test open ended

In [15]:
llm

In [6]:
from vllm import LLM

def echo_model_runner(self):
    return self.model_runner.model.__class__



print(llm.collective_rpc(echo_model_runner))

[<class 'vllm.compilation.cuda_graph.CUDAGraphWrapper'>]


In [7]:
def get_attr(self):
    return repr(self.model_runner.model)  # or other attributes

print(llm.collective_rpc(get_attr)[0])  # always get strings

In [8]:
# load test data
ds_test = datasets.load_dataset('json', data_files ='/root/autodl-fs/LRP/open_ended_dataset/all_data.json')
               
# calc ppl
model_list= [
    (llm, 'org_model'),
    (None, 'mask_en'),
    (None, 'mask_vi'),
    (None, 'mask_zh'),
]

data_list = [
    (ds_test, 'open_ended')    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    if model_name != 'org_model':
        i_model = get_mask_neuron_model_vllm(llm, activation_mask_path, model_name.split('_')[-1])

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        
        ans_list= get_open_ended_answer_vllm(i_model, list(i_data['train']['text']))#calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        # create model -col
        model_describe = [model_name+'|'+data_name]*len(ans_list)
        
        ds_test['train'] = ds_test['train'].add_column(name='describ', column=model_describe)        
        ds_test['train'] = ds_test['train'].add_column(name='ans', column=ans_list)        

ds_test['train'].to_json('~/autodl-fs/LRP/open_ended_data_generation/20251201_LAPE_org_generation.json', force_ascii=False)



********************
********************
model name: org_model
data name: open_ended
********************


Adding requests:   0%|          | 0/210 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/210 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

********************
********************
model name: mask_en
get mask =====lang:en=====
data name: open_ended
********************


Adding requests:   0%|          | 0/210 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/210 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

********************
********************
model name: mask_zh
get mask =====lang:zh=====
data name: open_ended
********************


Adding requests:   0%|          | 0/210 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/210 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

AttributeError: 'DatasetDict' object has no attribute 'to_json'

In [12]:
ds_test['train'].to_json('~/autodl-fs/LRP/open_ended_data_generation/20251201_LAPE_org_generation.json', force_ascii=False)



Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

35297

In [11]:
len(ds_test['train'])

210

In [5]:
# load test data
ds_test = datasets.load_dataset('json', data_files ='/root/autodl-fs/LRP/open_ended_dataset/all_data.json')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (None, 'mask_en'),
    (None, 'mask_vi'),
    (None, 'mask_zh'),
]

data_list = [
    (ds_test, 'open_ended')    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    if model_name != 'org_model':
        i_model = get_mask_neuron_model(model, activation_mask_path, model_name.split('_')[-1])

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        
        ans_list= get_open_ended_answer(i_model, tokenizer, list(i_data['train']['text']))#calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        # create model -col
        model_describe = [model_name+'|'+data_name]*len(ans_list)
        
        ds_test['train'].add_column(name='describ', column=model_describe)        
        ds_test['train'].add_column(name='ans', column=ans_list)        

ds_test.to_json('autodl-fs/LRP/open_ended_data_generation/20251201_LAPE_org_generation.json', force_ascii=False)



Generating train split: 0 examples [00:00, ? examples/s]

********************
********************
model name: org_model
data name: open_ended


  0%|          | 0/210 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


********************


  0%|          | 1/210 [00:08<30:33,  8.77s/it]

...

  4%|▍         | 8/210 [01:23<37:01, 11.00s/it]

itext: What are the major challenges faced by the education sector today?
content: Education is a fundamental right of every child, and it plays a crucial role in shaping their future. However, the education sector faces numerous challenges that hinder its ability to provide quality education to all. Here are some of the major challenges faced by the education sector today:
1. Access to Education: Despite progress made in recent years, millions of children around the world still lack access to basic education. Many rural areas, particularly in developing countries, have limited or no access to schools, making it difficult for children to receive an education.
2. Quality of Education: The quality of education varies greatly across different regions and countries. Many schools lack adequate resources, including textbooks, technology, and trained teachers, which can result in a subpar learning experience for students.
3. Teacher Shortages: Many countries face teacher shortages, particular

  4%|▍         | 9/210 [01:33<36:21, 10.85s/it]

itext: What are the primary factors that influence consumer behavior?
content: Consumer behavior is influenced by a variety of factors, including personal, psychological, social, cultural, and economic factors. Here are some of the primary factors that can impact consumer behavior:
1. Personal Factors: Consumers' personal characteristics such as age, income, occupation, lifestyle, values, attitudes, and beliefs play a significant role in shaping their buying habits. For instance, a young professional with a high income may be more likely to purchase luxury goods than an elderly person living on a fixed income.
2. Psychological Factors: Consumers' emotional states, motivations, and perceptions also affect their purchasing decisions. For example, a consumer who feels anxious or stressed may be more likely to seek comfort foods or indulge in retail therapy.
3. Social Factors: The opinions and behaviors of friends, family, and peers can significantly influence consumers' purchasing decisio

KeyboardInterrupt: 